Import Necessary Packages and Librarys 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import time

In [2]:
tf.__version__

'1.3.0'

Load The data

In [3]:
lines = open('cornell movie-dialogs corpus\movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [4]:
conversation_lines = open('cornell movie-dialogs corpus\movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [5]:
lines[:10]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?']

In [6]:
conversation_lines[:10]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']"]

Create a dictionary 

In [7]:
lineID_with_text = {}
for line in lines:
    line_number = line.split(' +++$+++ ')
    if len(line_number) == 5:
        lineID_with_text[line_number[0]] = line_number[4]
lineID_with_text

{'L1045': 'They do not!',
 'L1044': 'They do to!',
 'L985': 'I hope so.',
 'L984': 'She okay?',
 'L925': "Let's go.",
 'L924': 'Wow',
 'L872': "Okay -- you're gonna need to learn how to lie.",
 'L871': 'No',
 'L870': 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869': 'Like my fear of wearing pastels?',
 'L868': 'The "real you".',
 'L867': 'What good stuff?',
 'L866': "I figured you'd get to the good stuff eventually.",
 'L865': 'Thank God!  If I had to hear one more story about your coiffure...',
 'L864': "Me.  This endless ...blonde babble. I'm like, boring myself.",
 'L863': 'What crap?',
 'L862': 'do you listen to this crap?',
 'L861': 'No...',
 'L860': 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."',
 'L699': 'You always been this selfish?',
 'L698': 'But',
 'L697': "Then that's all you had to say.",
 'L696': 'Well, no...',
 'L695': "You never wanted to go out with 'me, did y

Create a list of all of the conversations' lines' ids.

In [8]:
conversation = []
for line in conversation_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversation.append(_line.split(','))
conversation

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366'],
 ['L367', 'L368'],
 ['L401', 'L402', 'L403'],
 ['L404', 'L405', 'L406', 'L407'],
 ['L575', 'L576'],
 ['L577', 'L578'],
 ['L662', 'L663'],
 ['L693', 'L694', 'L695'],
 ['L696', 'L697', 'L698', 'L699'],
 ['L860', 'L861'],
 ['L862', 'L863', 'L864', 'L865'],
 ['L866', 'L867', 'L868', 'L869'],
 ['L870', 'L871', 'L872'],
 ['L924', 'L925'],
 ['L984', 'L985'],
 ['L1044', 'L1045'],
 ['L49', 'L50', 'L51'],
 ['L571', 'L572', 'L573'],
 ['L579', 'L580'],
 ['L595', 'L596', 'L597'],
 ['L598', 'L599', 'L600'],
 ['L659', 'L660'],
 ['L952', 'L953'],
 ['L394', 'L395'],
 ['L396', 'L397'],
 ['L589', 'L590', 'L591'],
 ['L592', 'L593'],
 ['L756', 'L757', 'L758'],
 ['L759', 'L760'],
 ['L164', 'L165'],
 ['L319', 'L320'],
 ['L441', 'L442', 'L443', 'L444', 'L445']

In [9]:
questions = []
answers = []

for conv in conversation:
    for i in range(len(conv)-1):
        questions.append(lineID_with_text[conv[i]])
        answers.append(lineID_with_text[conv[i+1]])

In [10]:
questions

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "You're asking me out.  That's so cute. What's your name again?",
 "No, no, it's my fault -- we didn't have a proper introduction ---",
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Why?',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 'Gosh, if only we could find Kat a boyfriend...',
 "C'esc ma tete. This is my head",
 "Right.  See?  You're ready for the quiz.",
 "I don't want to know how to say that though.  I want to know useful things. Like where the good stores are.  How much does champagne cost?  Stuff like Chat.  I have n

In [11]:
for i in range(5):
    print(questions[i])
    print('********----------------------*********')
    print(answers[i])
    print('----------------------------------')

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
********----------------------*********
Well, I thought we'd start with pronunciation, if that's okay with you.
----------------------------------
Well, I thought we'd start with pronunciation, if that's okay with you.
********----------------------*********
Not the hacking and gagging and spitting part.  Please.
----------------------------------
Not the hacking and gagging and spitting part.  Please.
********----------------------*********
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?
----------------------------------
You're asking me out.  That's so cute. What's your name again?
********----------------------*********
Forget it.
----------------------------------
No, no, it's my fault -- we didn't have a proper introduction ---
********----------------------*********
Cameron.
----------------------------------


In [12]:
print(len(questions))
print(len(answers))

221616
221616


In [13]:
def clean_text(text):
    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [14]:
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))

In [15]:
for i in range(5):
    print(clean_questions[i])
    print('-------------------------------------------')
    print(clean_answers[i])
    print('-----------------*********-----------------')

can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again
-------------------------------------------
well i thought we would start with pronunciation if that is okay with you
-----------------*********-----------------
well i thought we would start with pronunciation if that is okay with you
-------------------------------------------
not the hacking and gagging and spitting part  please
-----------------*********-----------------
not the hacking and gagging and spitting part  please
-------------------------------------------
okay then how about we try out some french cuisine  saturday  night
-----------------*********-----------------
you are asking me out  that is so cute what is your name again
-------------------------------------------
forget it
-----------------*********-----------------
no no it is my fault  we did not have a proper introduction 
-------------------------------------------
cameron
--------

In [16]:
lengths = []
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))

lengths = pd.DataFrame(lengths, columns=['counts'])

In [17]:
lengths.describe()

,counts
count,443232.000000
mean,10.872094
std,12.215895
min,0.000000
25%,4.000000
50%,7.000000
75%,14.000000
max,555.000000


In [18]:
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

16.0
19.0
24.0
32.0
58.0


In [19]:
min_line_length = 2
max_line_length = 20

short_question_temp = []
short_answer_temp   = []

i=0
for ques in clean_questions:
    if len(ques.split()) >= min_line_length and len(ques.split()) <= max_line_length:
        short_question_temp.append(ques)
        short_answer_temp.append(clean_answers[i])
    i+=1

short_questions = []
short_answers = []

i=0
for ans in short_answer_temp:
    if len(ans.split()) >= min_line_length and len(ans.split()) <= max_line_length:
        short_answers.append(ans)
        short_questions.append(short_question_temp[i])
    i+=1
        

In [20]:
print("# of questions:", len(short_questions))
print("# of answers:", len(short_answers))
print("% of data used: {}%".format(round(len(short_questions)/len(questions),4)*100))

# of questions: 138335
# of answers: 138335
% of data used: 62.419999999999995%


In [21]:
vocabulary = {}

for ques in short_questions:
    for word in ques.split():
        if word not in vocabulary:
            vocabulary[word] = 1
        else:
            vocabulary[word] += 1

for ans in short_answers:
    for word in ans.split():
        if word not in vocabulary:
            vocabulary[word] = 1
        else:
            vocabulary[word] += 1

In [22]:
threshold =10
count =0
for w, n in vocabulary.items():
    if n>=threshold:
        count +=1
        

In [23]:
print("Size of total vocab:", len(vocabulary))
print("Size of vocab we will use:", count)

Size of total vocab: 45619
Size of vocab we will use: 8092


In [24]:
question_vocabulary_to_integer = {}
word_number = 0
for word, count in vocabulary.items():
    if count>=threshold:
        question_vocabulary_to_integer[word] = word_number
        word_number +=1
        
answer_vocabulary_to_integer = {}
word_number = 0
for word, count in vocabulary.items():
    if count>=threshold:
        answer_vocabulary_to_integer[word] = word_number
        word_number +=1

In [25]:
print(len(question_vocabulary_to_integer))
print(len(answer_vocabulary_to_integer))

8092
8092


In [26]:
len(answer_vocabulary_to_integer)

8092

In [27]:
codes = ['<PAD>','<EOS>','<UNK>','<GO>']

for code in codes:
    question_vocabulary_to_integer[code] = len(question_vocabulary_to_integer)+1
    
for code in codes:
    answer_vocabulary_to_integer[code] = len(answer_vocabulary_to_integer)+1

In [28]:
len(question_vocabulary_to_integer)

8096

In [29]:
question_vocabulary_to_integer['<PAD>']

8093

In [30]:
questions_integer_to_verb = {word_integer: word for word, word_integer in question_vocabulary_to_integer.items()}
answers_integer_to_verb =   {word_integer: word for word, word_integer in answer_vocabulary_to_integer.items()}

In [31]:
questions_integer_to_verb

{0: 'well',
 1: 'i',
 2: 'thought',
 3: 'we',
 4: 'would',
 5: 'start',
 6: 'with',
 7: 'if',
 8: 'that',
 9: 'is',
 10: 'okay',
 11: 'you',
 12: 'not',
 13: 'the',
 14: 'and',
 15: 'spitting',
 16: 'part',
 17: 'please',
 18: 'are',
 19: 'asking',
 20: 'me',
 21: 'out',
 22: 'so',
 23: 'cute',
 24: 'what',
 25: 'your',
 26: 'name',
 27: 'again',
 28: 'gosh',
 29: 'only',
 30: 'could',
 31: 'find',
 32: 'kat',
 33: 'a',
 34: 'boyfriend',
 35: 'ma',
 36: 'this',
 37: 'my',
 38: 'head',
 39: 'because',
 40: 'it',
 41: 'such',
 42: 'nice',
 43: 'one',
 44: 'how',
 45: 'our',
 46: 'little',
 47: 'date',
 48: 'plan',
 49: 'have',
 50: 'word',
 51: 'as',
 52: 'gentleman',
 53: 'do',
 54: 'get',
 55: 'hair',
 56: 'to',
 57: 'look',
 58: 'like',
 59: 'sure',
 60: 'really',
 61: 'wanna',
 62: 'go',
 63: 'but',
 64: 'cannot',
 65: 'unless',
 66: 'sister',
 67: 'goes',
 68: 'kind',
 69: 'of',
 70: 'guy',
 71: 'she',
 72: 'likes',
 73: 'pretty',
 74: 'ones',
 75: 'know',
 76: 'looked',
 77: 'for',

In [32]:
print(len(question_vocabulary_to_integer))
print(len(questions_integer_to_verb))
print(len(answer_vocabulary_to_integer))
print(len(answers_integer_to_verb))

8096
8096
8096
8096


In [33]:
for i in range(len(short_answers)):
    short_answers[i] +=' <EOS>'

In [34]:
short_answers

['not the hacking and gagging and spitting part  please <EOS>',
 'okay then how about we try out some french cuisine  saturday  night <EOS>',
 'forget it <EOS>',
 'let me see what i can do <EOS>',
 'right  see  you are ready for the quiz <EOS>',
 'forget french <EOS>',
 "well there's someone i think might be  <EOS>",
 'you are sweet <EOS>',
 "eber's deep conditioner every two days and i never ever use a blowdryer without the diffuser attachment <EOS>",
 'i really really really wanna go but i cannot  not unless my sister goes <EOS>',
 'i am working on it but she does not seem to be going for him <EOS>',
 'who knows  all i have ever heard her say is that she would dip before dating a guy that smokes <EOS>',
 'i believe we share an art instructor <EOS>',
 'i was <EOS>',
 "you never wanted to go out with 'me did you <EOS>",
 'then that is all you had to say <EOS>',
 'what crap <EOS>',
 'me  this endless blonde babble i am like boring myself <EOS>',
 'thank god  if i had to hear one more st

In [35]:
#just for testing
a='a am a i t k'
b={}
for i in a.split():
    if i not in b:
        b[i] = 1
    else:
        b[i] +=1 


In [36]:
b

{'a': 2, 'am': 1, 'i': 1, 'k': 1, 't': 1}

In [37]:
question_vocabulary_to_integer

{'well': 0,
 'i': 1,
 'thought': 2,
 'we': 3,
 'would': 4,
 'start': 5,
 'with': 6,
 'if': 7,
 'that': 8,
 'is': 9,
 'okay': 10,
 'you': 11,
 'not': 12,
 'the': 13,
 'and': 14,
 'spitting': 15,
 'part': 16,
 'please': 17,
 'are': 18,
 'asking': 19,
 'me': 20,
 'out': 21,
 'so': 22,
 'cute': 23,
 'what': 24,
 'your': 25,
 'name': 26,
 'again': 27,
 'gosh': 28,
 'only': 29,
 'could': 30,
 'find': 31,
 'kat': 32,
 'a': 33,
 'boyfriend': 34,
 'ma': 35,
 'this': 36,
 'my': 37,
 'head': 38,
 'because': 39,
 'it': 40,
 'such': 41,
 'nice': 42,
 'one': 43,
 'how': 44,
 'our': 45,
 'little': 46,
 'date': 47,
 'plan': 48,
 'have': 49,
 'word': 50,
 'as': 51,
 'gentleman': 52,
 'do': 53,
 'get': 54,
 'hair': 55,
 'to': 56,
 'look': 57,
 'like': 58,
 'sure': 59,
 'really': 60,
 'wanna': 61,
 'go': 62,
 'but': 63,
 'cannot': 64,
 'unless': 65,
 'sister': 66,
 'goes': 67,
 'kind': 68,
 'of': 69,
 'guy': 70,
 'she': 71,
 'likes': 72,
 'pretty': 73,
 'ones': 74,
 'know': 75,
 'looked': 76,
 'for': 77,

In [38]:
questions_int = []

for question in short_questions:
    ints = []
    for word in question.split():
        if word not in question_vocabulary_to_integer:
            ints.append(question_vocabulary_to_integer['<UNK>'])
        else:
            ints.append(question_vocabulary_to_integer[word])
    questions_int.append(ints)
    
answers_int=[]   
for answer in short_answers:
    ints = []
    for word in answer.split():
        if word not in answer_vocabulary_to_integer:
            ints.append(answer_vocabulary_to_integer['<UNK>'])
        else:
            ints.append(answer_vocabulary_to_integer[word])
    answers_int.append(ints)

In [39]:
print(len(questions_int))
print(len(answers_int))

138335
138335


In [40]:
word_count = 0
unk_count = 0

for question in questions_int:
    for word in question:
        if word == question_vocabulary_to_integer['<UNK>']:
            unk_count +=1
        word_count +=1
        
for answer in answers_int:
    for word in answer:
        if word == answer_vocabulary_to_integer['<UNK>']:
            unk_count +=1
        word_count +=1

unk_ratio = round(unk_count/word_count, 4)*100

print("Total number of words:", word_count)
print("Number of times <UNK> is used:", unk_count)
print("Percent of words that are <UNK>: {}%".format(round(unk_ratio,3)))

Total number of words: 2334533
Number of times <UNK> is used: 92441
Percent of words that are <UNK>: 3.96%


In [41]:
sorted_question = []
sorted_answer = []

for length in range(1, max_line_length+1):
    for i in enumerate(questions_int):
        if len(i[1]) == length:
            sorted_question.append(questions_int[i[0]])
            sorted_answer.append(answers_int[i[0]])
            
print(len(sorted_question))
print(len(sorted_answer))
print()
for i in range(55000,55500):
    print(sorted_question[i])
    print(sorted_answer[i])
    print()

138335
138335

[64, 33, 184, 83, 33, 8095]
[1, 53, 12, 106, 22, 8094]

[8095, 11, 1466, 83, 46, 8095]
[1, 90, 12, 1247, 119, 13, 46, 8094]

[1, 2, 40, 85, 37, 1031]
[40, 9, 1, 132, 54, 36, 11, 62, 1058, 8094]

[3, 367, 8, 43, 291, 4401]
[323, 17, 53, 12, 62, 173, 56, 13, 3454, 56, 372, 25, 575, 8094]

[86, 8095, 3166, 8, 9, 951]
[332, 132, 99, 83, 102, 553, 8094]

[332, 132, 99, 83, 102, 553]
[996, 612, 100, 22, 214, 1115, 5645, 8094]

[71, 936, 404, 103, 13, 135]
[71, 936, 404, 297, 13, 4377, 442, 8094]

[125, 0, 1208, 77, 13, 1177]
[11, 18, 1472, 1, 85, 134, 195, 56, 186, 11, 7, 11, 4, 58, 56, 1018, 642, 8094]

[3, 467, 56, 8095, 13, 1253]
[661, 1303, 103, 2839, 8094]

[2724, 24, 267, 11, 271, 8]
[25, 618, 8095, 208, 25, 433, 8094]

[25, 618, 8095, 208, 25, 433]
[134, 13, 170, 1000, 612, 8094]

[3, 132, 327, 448, 69, 40]
[14, 44, 630, 159, 11, 271, 11, 174, 77, 36, 24, 159, 11, 483, 40, 33, 4487, 4488, 8094]

[359, 8, 3, 18, 195, 225]
[811, 1250, 165, 475, 45, 7054, 78, 103, 45, 3387

[429, 297, 8095, 25, 2869, 8095, 8094]

[24, 9, 470, 69, 37, 731]
[106, 1, 132, 49, 33, 2778, 8094]

[106, 1, 132, 49, 33, 2778]
[558, 296, 157, 491, 211, 8094]

[42, 2735, 277, 11, 2089, 40]
[346, 13, 5057, 2435, 685, 11, 111, 8094]

[125, 37, 126, 1, 57, 1358]
[11, 49, 76, 33, 880, 944, 358, 473, 8094]

[5077, 106, 11, 18, 103, 152]
[1, 53, 12, 75, 12, 33, 732, 653, 40, 9, 609, 33, 33, 8095, 781, 8094]

[242, 242, 1, 367, 119, 8]
[457, 225, 1, 3374, 13, 1749, 1, 90, 33, 8095, 8095, 111, 8094]

[1, 142, 350, 8095, 350, 8095]
[153, 1002, 20, 8, 9, 24, 1, 90, 248, 77, 56, 899, 56, 2395, 8094]

[153, 160, 159, 11, 686, 488]
[40, 9, 25, 224, 1882, 46, 66, 8094]

[1091, 8095, 2444, 8095, 12, 679]
[1, 90, 834, 56, 352, 11, 49, 12, 345, 25, 2103, 69, 3818, 8094]

[134, 33, 46, 3548, 2064, 4688]
[36, 880, 786, 9, 33, 3548, 8094]

[36, 880, 786, 9, 33, 3548]
[641, 1782, 17, 8094]

[11, 4179, 266, 56, 268, 8095]
[14, 99, 4179, 20, 56, 843, 8094]

[14, 99, 4179, 20, 56, 843]
[153, 160, 8095, 77,